In [1]:
import os
import sys

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant

import numpy as np
import csv
import pandas as pd

## Prepare data

In [2]:
with open ("../data/ml-100k/u.data", "r") as f:
    data = list(csv.reader(f, delimiter="\t"))
data = np.array(data)
film_dim = np.amax(np.array(data[:,1]).astype(np.float))
user_dim = np.amax(np.array(data[:,0]).astype(np.float))
print("film size", film_dim)
print("user size", user_dim)
data = data.astype(np.int)
data = pd.DataFrame(data)
data.sort_values([0,3],inplace=True) ## Sort data
data.rename(columns= {0: 'user_id',
                      1: 'movie_id',
                      2: 'rating',
                      3: 'time'},
            inplace=True)
data_x = []
data_y = []
max_len = 0
for i in range(1, user_dim.astype(np.int)+1):
    user = data[data.user_id == i]
    if len(user) > max_len:
        max_len = len(user)
    data_x.append(user['movie_id'])
    data_y.append(user['rating'])

data_x =  tf.keras.preprocessing.sequence.pad_sequences(data_x,padding='post',maxlen=max_len)
data_y =  tf.keras.preprocessing.sequence.pad_sequences(data_y,padding='post',maxlen=max_len)

train_x = data_x[:700]
test_x = data_x[700:]
train_y = data_y[:700]
test_y = data_y[700:]

film size 1682.0
user size 943.0


### Embeddings size
While LSTM units = 64 and movies were not order by time of rating

| Hyperparamater-value | Accuracy | Test Accuracy |
| -------------------- | -------- | ------------- |
| 64                   | 46.94%   | 35.22%        |
| 128                  | 50.09%   | 34.63%        |
| 256                  | 53.71%   | 34.13%        |
| 512                  | 58.82%   | 34.34%        |

### LSTM Units
While embedding size was 300 and movies were not order by time of rating

| Hyperparamater-value | Accuracy | Test Accuracy |
| -------------------- | -------- | ------------- |
| lstm_units (32)      | 51.75%   | 33.73%        |
| lstm_units (64)      | 53.68%   | 33.22%        |
| lstm_units (128)     | 53.52%   | 33.76%        |

### Sort data by time

LSTM unit = 64
Embeddings size = 512
Accuracy: 63.40%
Test Accuracy: 32.43%

### Adding droput layer

We have tried to add droput layer for our model to combat some overtraining in case there is some. But with parameters of LSTM units (64) and embbedings size (128) we didnt see any improvement:

Accuracy was 51.64% and validadtion accuracy: was 34.92%.
Therefore there are no significant improvements by adding Dropout(0.5) layer 

In [19]:
class POSTagger(keras.Model):

    def __init__(self):
        super(POSTagger, self).__init__()
        lstm_units = 64
        num_classes = 6
        self.emb = Embedding(
            input_dim=1683 + 1,
            output_dim=128,
            mask_zero=True,
            trainable=True)

        self.lstm = Bidirectional(LSTM(
            units=lstm_units, return_sequences=True))
        
        self.dropout = Dropout(0.5)
        
        self.dense = Dense(
            units=num_classes,
            activation='softmax')
        
        

    def call(self, x):
        mask = self.emb.compute_mask(x)
        x = self.emb(x)
        x = self.lstm(x, mask=mask)
        x = self.dropout(x)
        x = self.dense(x)
        return x


model = POSTagger()

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(
    x=train_x,
    y=train_y,
    batch_size=10,
    epochs=30,
    validation_data=(test_x, test_y))

Train on 700 samples, validate on 243 samples
Epoch 1/30
700/700 [==============================] - 37s 53ms/sample - loss: 0.2263 - accuracy: 0.3281 - val_loss: 0.1890 - val_accuracy: 0.3487
Epoch 2/30
700/700 [==============================] - 28s 40ms/sample - loss: 0.2089 - accuracy: 0.3615 - val_loss: 0.1828 - val_accuracy: 0.3743
Epoch 3/30
700/700 [==============================] - 28s 40ms/sample - loss: 0.2041 - accuracy: 0.3801 - val_loss: 0.1817 - val_accuracy: 0.3789
Epoch 4/30
700/700 [==============================] - 28s 40ms/sample - loss: 0.2022 - accuracy: 0.3874 - val_loss: 0.1818 - val_accuracy: 0.3777
Epoch 5/30
700/700 [==============================] - 28s 40ms/sample - loss: 0.1996 - accuracy: 0.3976 - val_loss: 0.1814 - val_accuracy: 0.3779
Epoch 6/30
700/700 [==============================] - 28s 40ms/sample - loss: 0.1977 - accuracy: 0.4038 - val_loss: 0.1843 - val_accuracy: 0.3679
Epoch 7/30
700/700 [==============================] - 28s 40ms/sample - loss: 

Save weights of the model

In [45]:
model.save_weights("../models/LSTM", save_format="tf")

Since subclassed keras models canot be saved first we have to create and compile one and then we can load weights:

In [46]:
sample_model = POSTagger()
sample_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

sample_model.load_weights("../models/LSTM")
pred = sample_model.predict(test_x[0], verbose=0)

### Example of prediction of our model

we can see that for first user we are able to predict rating for each of this users movies. Since there is 5 star rating (+ 1 for not rated movie) we get array with prediction of 6 of those ratings.

In [47]:
pred

array([[[0.01448555, 0.05996639, 0.05869912, 0.21839991, 0.3288063 ,
         0.31964275]],

       [[0.03245169, 0.10657497, 0.14628832, 0.17322175, 0.26685464,
         0.27460858]],

       [[0.07072053, 0.11664139, 0.15948306, 0.19422805, 0.23257437,
         0.22635266]],

       ...,

       [[0.13320369, 0.15760657, 0.160003  , 0.18471362, 0.19620894,
         0.16826418]],

       [[0.13320369, 0.15760657, 0.160003  , 0.18471362, 0.19620894,
         0.16826418]],

       [[0.13320369, 0.15760657, 0.160003  , 0.18471362, 0.19620894,
         0.16826418]]], dtype=float32)